# Exploring Feature Data for Full Model Training

**Goal:** Verify all 20 engineered features are available and prepare for training with the complete feature set.

## What We're Testing:
1. Load Parquet features from S3
2. Verify all 20 feature columns exist
3. Check data quality
4. Prepare parameters for pipeline execution

In [ ]:
import boto3
import pandas as pd
import sagemaker

# Initialize SageMaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
region = sess.boto_region_name

print(f"Region: {region}")
print(f"Default Bucket: {bucket}")

## 1. Load Training Features from S3

In [ ]:
# Public bucket where features are stored
PUBLIC_BUCKET = 'sagemaker-us-east-1-425709451100'
FEATURES_PREFIX = 'aai540-group1/features'

# Load training features
train_path = f"s3://{PUBLIC_BUCKET}/{FEATURES_PREFIX}/train_features.parquet"
print(f"Loading: {train_path}")

df_train = pd.read_parquet(train_path)

print(f"\nLoaded {len(df_train):,} rows × {df_train.shape[1]} columns")
print(f"\nColumns: {list(df_train.columns)}")

## 2. Verify All 20 Feature Columns

In [ ]:
# Define all 20 engineered features (including target-encoded rates)
ALL_FEATURES = [
    # Temporal (9)
    'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEP_HOUR', 'SCHEDULED_DEPARTURE',
    'HOUR_SIN', 'HOUR_COS', 'IS_PEAK_HOUR', 'IS_WEEKEND',
    
    # Distance (4)
    'DISTANCE', 'SCHEDULED_TIME', 'IS_LONG_HAUL', 'DISTANCE_BUCKET',
    
    # Target-encoded (4) - THESE WERE EXCLUDED IN ENGINEERED BASELINE
    'AIRLINE_DELAY_RATE', 'ORIGIN_DELAY_RATE', 'DEST_DELAY_RATE', 'ROUTE_DELAY_RATE',
    
    # Volume (3)
    'ORIGIN_FLIGHTS', 'DEST_FLIGHTS', 'ROUTE_FLIGHTS'
]

TARGET = 'DELAYED'

print(f"Expected features: {len(ALL_FEATURES)}")
print(f"\nChecking for all features...")

missing_features = [f for f in ALL_FEATURES if f not in df_train.columns]
if missing_features:
    print(f"❌ Missing features: {missing_features}")
else:
    print(f"✅ All {len(ALL_FEATURES)} features are present!")

# Check target column
if TARGET in df_train.columns:
    print(f"✅ Target column '{TARGET}' found")
else:
    print(f"❌ Target column '{TARGET}' missing")

## 3. Inspect Data Quality

In [ ]:
# Select only the features and target we need
df_model = df_train[ALL_FEATURES + [TARGET]].copy()

print(f"Model dataset: {df_model.shape[0]:,} rows × {df_model.shape[1]} columns")
print(f"\nData types:")
print(df_model.dtypes)

print(f"\nMissing values:")
missing = df_model.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("No missing values ✅")

print(f"\nTarget distribution:")
delay_rate = df_model[TARGET].mean() * 100
print(f"Delayed: {(df_model[TARGET]==1).sum():,} ({delay_rate:.2f}%)")
print(f"On-time: {(df_model[TARGET]==0).sum():,} ({100-delay_rate:.2f}%)")

## 4. Sample Data Preview

In [ ]:
print("First 5 rows (all 20 features + target):")
df_model.head()

## 5. Compare with Engineered Baseline (16 Features)

The engineered baseline excluded these 4 target-encoded features:

In [ ]:
# Features that were EXCLUDED in engineered baseline
EXCLUDED_IN_BASELINE = [
    'AIRLINE_DELAY_RATE', 
    'ORIGIN_DELAY_RATE', 
    'DEST_DELAY_RATE', 
    'ROUTE_DELAY_RATE'
]

print("Target-encoded features (excluded in baseline, included now):")
print(df_model[EXCLUDED_IN_BASELINE].describe())

print("\n📊 These 4 features capture historical delay patterns:")
print("  - AIRLINE_DELAY_RATE: Historical delay rate for this airline")
print("  - ORIGIN_DELAY_RATE: Historical delay rate at origin airport")
print("  - DEST_DELAY_RATE: Historical delay rate at destination airport")
print("  - ROUTE_DELAY_RATE: Historical delay rate for this specific route")

## 6. Pipeline Parameters for Full Model Training

Based on `02_engineered_baseline.ipynb` hyperparameters:

In [ ]:
# Hyperparameters from engineered baseline
HYPERPARAMETERS = {
    'Objective': 'binary:logistic',
    'EvalMetric': 'auc',
    'MaxDepth': 8,
    'Eta': 0.05,
    'NumRound': 1000,
    'ScalePosWeight': 4.58,
    'Subsample': 0.8,
    'ColsampleByTree': 0.8,
    'MinChildWeight': 1
}

# Data paths (Parquet format)
DATA_PATHS = {
    'TrainingDataUrl': f's3://{PUBLIC_BUCKET}/{FEATURES_PREFIX}/train/',
    'ValidationDataUrl': f's3://{PUBLIC_BUCKET}/{FEATURES_PREFIX}/validation/',
    'InputContentType': 'application/x-parquet'
}

print("🚀 Pipeline Parameters for Full Model (20 features):")
print("\nHyperparameters:")
for k, v in HYPERPARAMETERS.items():
    print(f"  --{k} {v}")

print("\nData Configuration:")
for k, v in DATA_PATHS.items():
    print(f"  --{k} {v}")

## 7. Command to Run Pipeline

**Note:** The pipeline currently expects CSV format. You'll need to either:
1. Use the CSV files prepared in `02_prepare_sagemaker_data.ipynb`, OR
2. Prepare Parquet files with only the 20 features + target (no metadata columns)

In [ ]:
# Check if CSV training data exists
s3_client = boto3.client('s3')

csv_paths = [
    f'{bucket}/aai540-group1/training/engineered-baseline/train/train.csv',
    f'{bucket}/aai540-group1/training/engineered-baseline/val/val.csv'
]

print("Checking for prepared CSV training data...")
for path in csv_paths:
    key = path.split('/', 1)[1]
    try:
        s3_client.head_object(Bucket=bucket, Key=key)
        print(f"  ✅ Found: s3://{path}")
    except:
        print(f"  ❌ Missing: s3://{path}")
        print(f"     Run: notebooks/02_feature_engineering/02_prepare_sagemaker_data.ipynb")

## Next Steps

1. **If CSV data exists:** Run the pipeline with the command below
2. **If CSV data missing:** Run `02_prepare_sagemaker_data.ipynb` first to create CSV files

### Command to Run Pipeline (from terminal):

```bash
cd /Users/arr/USD/AAI-540/aai540_group1/tmp/scripts_v2

python run_experiment_v2.py \
    --TrainingDataUrl s3://YOUR-BUCKET/aai540-group1/training/engineered-baseline/train/ \
    --ValidationDataUrl s3://YOUR-BUCKET/aai540-group1/training/engineered-baseline/val/ \
    --InputContentType text/csv \
    --MaxDepth 8 \
    --Eta 0.05 \
    --NumRound 1000 \
    --ScalePosWeight 4.58 \
    --display-name "full-engineered-20-features"
```